# Проект: Обучение с учителем: качество модели

**Описание проекта**

Интернет-магазин «В один клик» продаёт разные товары: для детей, для дома, мелкую бытовую технику, косметику и даже продукты. Отчёт магазина за прошлый период показал, что активность покупателей начала снижаться. Привлекать новых клиентов уже не так эффективно: о магазине и так знает большая часть целевой аудитории. Возможный выход — удерживать активность постоянных клиентов. Сделать это можно с помощью персонализированных предложений.

«В один клик» — современная компания, поэтому её руководство не хочет принимать решения просто так — только на основе анализа данных и бизнес-моделирования.

**Цель исследования** - разработать решение, которое позволит персонализировать предложения постоянным клиентам, чтобы увеличить их покупательскую активность.

**Описание данных**

market_file.csv - таблица, которая содержит данные о поведении покупателя на сайте, о коммуникациях с покупателем и его продуктовом поведении.

market_money.csv - таблица с данными о выручке, которую получает магазин с покупателя, то есть сколько покупатель всего потратил за период взаимодействия с сайтом.

market_time.csv - таблица с данными о времени (в минутах), которое покупатель провёл на сайте в течение периода.

money.csv - таблица с данными о среднемесячной прибыли продавца за последние 3 месяца: какую прибыль получает магазин от продаж каждому покупателю.

**Этапы проекта:**
1. Загрузка данных
2. Предобработка данных
3. Исследовательский анализ
4. Объединение таблиц
5. Корреляционный анализ
6. Использоваине пайплайнов
7. Анализ важности признаков
8. Сегментация покупателей
9. Общий вывод

## Загрузка данных

**Импортируем библиотеки**

In [4]:
!pip install beautifulsoup4==4.9.3
!pip install matplotlib==3.3.4
!pip install nltk==3.6.1
!pip install numpy==1.20.1
!pip install pandas==1.2.4
!pip install plotly==5.4.0
!pip install psycopg2-binary==2.9.2
!pip install regex==2022.3.15
!pip install scikit-learn==0.24.1
!pip install scipy==1.8.0
!pip install seaborn==0.11.1
!pip install sqlalchemy==1.4.15
!pip install statsmodels==0.13.2
!pip install shap

     |████████████████████████████████| 999 kB 2.4 MB/s eta 0:00:01


In [2]:
#!pip install --upgrade scikit-learn

In [5]:
# Для работы с данными
import pandas as pd
import numpy as np

# Для интерпретации результатов машинного обучения
import shap

# Для корреляционного анализа
import phik

# Построение графиков
import seaborn as sns
import matplotlib.pyplot as plt

# Функция для разделения данных
from sklearn.model_selection import (train_test_split,  # Функция для разделения данных
                                    RandomizedSearchCV) # Импортируем класс RandomizedSearchCV

# Для оценки мультиколлинеарности
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

# Загружаем нужные классы
# Класс ColumnTransformer помогает работать с данными разного типа в одном наборе
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# дополнительные классы для преобразования данных
from sklearn.preprocessing import (OneHotEncoder,
                                   OrdinalEncoder,
                                   StandardScaler,
                                   MinMaxScaler,
                                   RobustScaler)

# Алгоритмы машинного обучения
from sklearn.linear_model import LogisticRegression # Метод бинарной классификации
from sklearn.neighbors import KNeighborsClassifier  # Метод К ближайших соседей
from sklearn.svm import SVC                         # Метод опорных векторов
from sklearn.tree import (DecisionTreeClassifier,   # Модель дерева решения
                         plot_tree)                 # Визуализация модели дерева решения

# Класс для работы с пропусками
from sklearn.impute import SimpleImputer

# Загружаем функцию для работы с метриками
from sklearn.metrics import (roc_auc_score,
                             f1_score)


# Обявляем константы
RANDOM_STATE = 100
TEST_SIZE = 0.25

ModuleNotFoundError: No module named 'phik'

**Загружаем данные**

In [ ]:
market_file = pd.read_csv('/datasets/market_file.csv')
market_money = pd.read_csv('/datasets/market_money.csv')
market_time = pd.read_csv('/datasets/market_time.csv')
money = pd.read_csv('/datasets/money.csv')

**Функция вывода общей информации**

In [ ]:
def general_information(data):
    display('Вывод первых строк')
    display(data.head())
    display('Вывод общей информации')
    display(data.info())
    display('Количество пропусков')
    display(data.isna().sum())
    display('Количество дубликатов')
    display(data.duplicated().sum())

### market_file

In [ ]:
general_information(market_file)

In [ ]:
market_file.columns

Переменнуем название столбцов, заменим пробелы на "_". 

Название оставим на кириллице. 

In [ ]:
market_file = market_file.rename(
    columns={
        'Покупательская активность':'Покупательская_активность',
        'Тип сервиса':'Тип_сервиса',
        'Разрешить сообщать':'Разрешить_сообщать'
    }
)

market_file.columns

### market_money

In [ ]:
general_information(market_money)

### market_time

In [ ]:
general_information(market_time)

Запишем название столбца с заглавной буквы.

In [ ]:
market_time = market_time.rename(
    columns={
        'минут':'Минут'
    }
)

market_time.columns

### money

In [ ]:
general_information(money)

Разделим столбцы, в прибыли заменим запятую на точку и приведём тип данных к флоат.

In [ ]:
money = pd.read_csv('/datasets/money.csv', sep=';', decimal=',')
general_information(money)

### Вывод

В работу поступило 4 датасета
- market_file - 13 столбцов, 1300 строк, 0 пропусков, 0 дубликатов;
- market_money - 3 столбцов, 3900 строк, 0 пропусков, 0 дубликатов;
- market_time - 3 столбцов, 2600 строк, 0 пропусков, 0 дубликатов;
- money - 2 столбцов, 1300 строк, 0 пропусков, 0 дубликатов.

В market_file и market_time отредактированы название столбцов. В money произведено разделение столбцов и проведена замена тип столбца "Прибыль" на float64.

Во всех четырёх датасетах название столбцов соотвествуют описанию. 

Замечены опечатки в названиях данных их редактирование будет произведенно на этапе предобработки данных.

## Предобработка данных

**Функция проверки данных на выбросы.**

In [ ]:
def emissions_df(data, column):
    # Описательная статистика
    display(data[column].describe())
    # Диаграмма размаха
    data.boxplot(column=column)
    plt.title('Диаграмма размаха')
    plt.show()

### market_file

Проверим уникальные значения данных.

In [ ]:
for i in market_file:
    display(market_file[i].unique())

Название строковых данных пропишем строковыми буквами. Данные "стандартт" заменим на "стандарт".

In [ ]:
market_file.columns = market_file.columns.str.lower()
market_file = market_file.applymap(lambda x: x.lower() if isinstance(x, str) else x)
market_file['тип_сервиса'] = market_file['тип_сервиса'].replace('стандартт', 'стандарт')

for i in market_file:
    display(market_file[i].unique())
    
display(market_file.columns)

Проверка на выбросы.

In [ ]:
market_file.head()

In [ ]:
emissions_df(market_file, 'маркет_актив_6_мес')

In [ ]:
emissions_df(market_file, 'длительность')

In [ ]:
emissions_df(market_file, 'акционные_покупки')

Исправлена опечатка в обозначении данных на "стандарт". В столбце "Длительность" данные без выбросов, в столбцах "Акционные_покупки" и "Маркет_актив_6_мес" имеются выбросы, принято решение их не удалять т.к. их достаточно много их влияние оценим на следующем шаге.

### market_money

Проверим уникальные значения данных.

In [ ]:
for i in market_money:
    display(market_money[i].unique())

Название строковых данных пропишем строковыми символами.

In [ ]:
market_money = market_money.applymap(lambda x: x.lower() if isinstance(x, str) else x)
market_money.columns = market_money.columns.str.lower()

for i in market_money:
    display(market_money[i].unique())
    
display(market_money.columns)

Проверка на выбросы.

In [ ]:
emissions_df(market_money, 'выручка')

Значение выручки в 106862.2 является явным выбросом, удалим его. Т.к. нам в следующем шаге понадобятся только те клиенты которые покупали значение ноль тоже удалим.

In [ ]:
market_money = market_money.loc[market_money['выручка'] != 106862.2]
market_money = market_money.loc[market_money['выручка'] != 0]
# Проверим удаление данных
emissions_df(market_money, 'выручка')

Были удаленны два значения 106862.2 - это явный выброс и 0 - это значение удаленно по условиям исследования. Так же большое колличество выбросов которые не будут удаленны.

### market_time

Название строковых данных пропишем строковыми символами.

In [ ]:
market_time = market_time.applymap(lambda x: x.lower() if isinstance(x, str) else x)
market_time.columns = market_time.columns.str.lower()

for i in market_time:
    display(market_time[i].unique())
    
display(market_time.columns)

Проверим уникальные значения данных.

In [ ]:
for i in market_time:
    display(market_time[i].unique())

Исправим опечатку в названии данных 'предыдцщий_месяц' на 'предыдущий_месяц' и сделаем их с заглавной буквы.

In [ ]:
market_time['период'] = market_time['период'].replace('предыдцщий_месяц', 'предыдущий_месяц')
market_time = market_time.applymap(lambda x: x.capitalize() if isinstance(x, str) else x)

Проверка на выбросы.

In [ ]:
emissions_df(market_time, 'минут')

Исправленна опечатка в данных на 'предыдущий_месяц'. Данные без выбросов.

### money

Название строковых данных пропишем строковыми символами. Проверим уникальные значения данных.

In [ ]:
money = money.applymap(lambda x: x.lower() if isinstance(x, str) else x)
money.columns = money.columns.str.lower()

for i in money:
    display(money[i].unique())
    
display(money.columns)

Проверка на выбросы.

In [ ]:
emissions_df(money, 'прибыль')

Значения прибыли имеют выбросы их удалять не будем.

### Вывод

Было выполнено:
- market_file - Исправлена опечатка в обозначении данных на "стандарт". В столбце "Длительность" данные без выбросов, в столбцах "Акционные_покупки" и "Маркет_актив_6_мес" имеются выбросы, принято решение их не удалять т.к. их достаточно много их влияние оценим на следующем шаге.
- market_money - Были удаленны два значения 106862.2 - это явный выброс и 0 - это значение удаленно по условиям исследования. Так же большое колличество выбросов которые не будут удаленны.
- market_time - Исправленна опечатка в данных на 'предыдущий_месяц'. Данные без выбросов.
- money - Значения прибыли имеют выбросы их удалять не будем,
- название столбцов были проведены к змеевидному виду.

## Исследовательский анализ данных

**Функция построения круговой диаграммы для категориальных данных.**

In [ ]:
def cat_pie(data, column):
    (
        data[column]
        .value_counts()
        .plot(kind='pie', autopct='%1.0f%%', figsize=(7, 7), label='')
    )
    plt.title(f'Доля клиентов по {column}')
    plt.show()

**Функция построения гистограммы для двух столбцов.**

In [ ]:
def num_hist(data, column_1, column_2, bins):
    sns.set()
    plt.figure(figsize=(10, 10))
    if column_2 == None:
        sns.histplot(data, bins=bins, kde=True, x=column_1)
    else:
        sns.histplot(data, bins=bins, kde=True, hue=column_2, x=column_1)
    plt.title(f'Распределение признака {column_1}')
    plt.xlabel(column_1)
    plt.ylabel('Количество')dd
    plt.show()

**Функция построения гистограммы для категориальных переменных.**

In [ ]:
def count_hist(data, column_1, column_2):
    sns.set()
    plt.figure(figsize=(15, 10))
    sns.countplot(data=data, x=column_1, hue=column_2) 
    plt.title(f'Распределение признака {column_1}')
    plt.xlabel(column_1)
    plt.ylabel('Количество')
    plt.show()

### market_file

Строим круговые диаграммы для категориальных данных.

In [ ]:
cat_pie(market_file, 'покупательская_активность')

Большинство клиентов, 62%, сохранило свою активность. У 38% клиентов активность снизилась, это довольно весомая часть клиентов.

In [ ]:
cat_pie(market_file, 'тип_сервиса')

Количество клиентов с премиумным сервисом 29%.

In [ ]:
cat_pie(market_file, 'разрешить_сообщать')

Большинство клиентов, 74%, хотят получать дополнительные предложения о товарах.

In [ ]:
cat_pie(market_file, 'популярная_категория')

Четверть всех покупок занимает товары для детей, какую то категорию выделить невозможно.

Построение диаграммы.

In [ ]:
num_hist(market_file, 'маркет_актив_6_мес', 'покупательская_активность', 30)

Маркетинговая активность для клиентов со сниженной покупательской активностью распределенны болле менне на одном уровне в течении всех 6 месяцев, со сплеском в период с 3 по 4 месяц.

Маркетинговая активность для клиентов с прежним уровнем покупательской активностью показали свою активность с 3 по 6 месяц, с 1 по 3 месяц таких клиентов практически нет.

In [ ]:
count_hist(market_file, 'маркет_актив_тек_мес', 'покупательская_активность')

Больше всех рекламу получают клиенты с прежней покупательской активностью.

In [ ]:
num_hist(market_file, 'длительность', 'покупательская_активность', 40)

Имеются выбросы по двум категориям, скорее всего, они указывают на регистрацию клиентов в момент каких либо акций или проведении рекламной компании интернет магазина.

In [ ]:
num_hist(market_file, 'акционные_покупки', 'покупательская_активность', 30)

Клиенты со сниженной покупательской активностью чаще покупают товары по акции.

In [ ]:
market_file['средний_просмотр_категорий_за_визит'].unique()

In [ ]:
count_hist(market_file, 'средний_просмотр_категорий_за_визит', 'покупательская_активность')

Клиенты с прежней покупательской способностью просматривают сразу несколько категорий товаров. А те у кого снизилась покупательская способность смотрят в основном до 3 категорий, что указывает на просмотр конкретных товаров, скорее всего по акции.

In [ ]:
count_hist(market_file, 'неоплаченные_продукты_штук_квартал', 'покупательская_активность')

Количество неоплаченных товаров уменьшается с их количеством, что указывает, скорее всего, на то, что клиенты набирают корзину и только когда наберётся 6 и более товаров оплачивают и забирают их.

In [ ]:
count_hist(market_file, 'ошибка_сервиса', 'покупательская_активность')

У большинства клиентов количество ошибок от трёх до шести.

In [ ]:
count_hist(market_file, 'страниц_за_визит', 'покупательская_активность')

Таже картина, что просмотр категорий за визит.

### market_money

Строим круговые диаграммы для категориальных данных.

In [ ]:
cat_pie(market_money, 'период')

Подсчитаем выручку за каждый из трёх месяцев.

In [ ]:
market_money_period = (
    market_money
    .pivot_table(index='период', values='выручка', aggfunc='sum')
    .sort_values(by='выручка', ascending=False)
)
market_money_period

In [ ]:
cat_pie(market_money_period, 'выручка')

Построение диаграммы.

In [ ]:
num_hist(market_money, 'выручка', None, 30)

Выручка магазина растёт от в течении последних трёх месяцев.

Диаграмма выручки выглядит нормально.

### market_time

Строим круговые диаграммы для категориальных данных.

Подсчитаем количество минут по каждому месяцу.

In [ ]:
market_time_period = (
    market_time
    .pivot_table(index='период', values='минут', aggfunc='sum')
    .sort_values(by='минут', ascending=False)
)
market_time_period

In [ ]:
cat_pie(market_time_period, 'минут')

Построение диаграммы.

In [ ]:
num_hist(market_time, 'минут', None, 20)

Клиенты провели времени, примерно, одинаково в двух последних месяцах.

### money

Построение диаграммы.

In [ ]:
num_hist(money, 'прибыль', None, 30)

Значения признака прибыль распределены нормально.

### Выделяем клиентов с покупательской способностью не менее трёх мясецев.

In [ ]:
# Группируем клиентов по id
# Применяем агрегатную функцию count по группе 'Период'
month_3 = market_money.groupby('id').agg({'период': 'count'})

# Оставляем клиентов которые активны все 3 месяца
month_3 = month_3.loc[month_3['период'] == 3]

# Применяем фильтр данных isin к market_file по значением из month_3
market_file = market_file[market_file['id'].isin(month_3.index)]

display(f'Количество клиентов, чья активность 3 и более месяцев {len(market_file)}')

### Вывод

Обобщая информацию по всем 4 датафреймам можно увидеть, что даже при снижении маркетинговой активности время пользователей в магазине не изменилось, выручка при этом расла последнии 3 месяца. 

Пользователи со сниженной покупательской активностью просматривают страниц и категорий товаров одинаково, что даёт нам понять, что они ищут товары только по акции. Они же получают всех больше рекламных уведомлений от магазина.

Пользователи с прежней активностью просматривают большое количество страниц в магазине и различные категории, и отправляют товары в корзину и оплачивают их когда в корзине становится 6 и более товаров.

По итогу 1296 клиентов сохраняло свою активность на протяжении не менее трёх месяцев.

## Объединение таблиц

In [ ]:
# Создаём сводную таблицу
market_money_group = market_money.pivot_table(index='id', columns='период')
# Переименовываем столбцы
market_money_group.columns = ['выручка_предыдущий_месяц',
                              'выручка_препредыдущий_месяц',
                              'выручка_текущий_месяц']
# Создаём столбец id 
#market_money_group['id'] = market_money_group.index

# Создаём сводную таблицу
market_time_group = market_time.pivot_table(index='id', columns='период')
# Переименовываем столбцы
market_time_group.columns = ['минут_предыдущий_месяц',
                             'минут_текущий_месяц']

# Создаём итоговый датафрейм объединяя market_file и market_money_group
market_result = market_file.join(market_money_group, on='id')

# Объединяем датафреймы market_result и market_time_group
market_result = market_result.join(market_time_group, on='id')

# Выводим общую информацию
general_information(market_result)

### Вывод

Итоговая таблица market_result состоящая из market_file, market_money, market_time:
- 18 столбцов,
- 1296 записей,
- пропуски отсуствуют,
- дубликаты отсуствуют.

## Корреляционный анализ

In [ ]:
# Создаём переменную для корреляционного анализа
# Удаляем столбец c id, т.к. он ни на, что не влияет
# Строим матрицу корреляции для числовых столбцов
quantity_phik = (
    market_result
    .drop('id', axis=1)
    .phik_matrix(interval_cols=['маркет_актив_6_мес',
                               'маркет_актив_тек_мес',
                               'длительность',
                               'акционные_покупки',
                               'средний_просмотр_категорий_за_визит',
                               'неоплаченные_продукты_штук_квартал',
                               'ошибка_сервиса',
                               'страниц_за_визит',
                               'выручка_предыдущий_месяц',
                               'выручка_препредыдущий_месяц',
                               'выручка_текущий_месяц',
                               'минут_предыдущий_месяц',
                               'минут_текущий_месяц'])
)

# Выводим корреляционную матрицу
(
    quantity_phik
    .style.background_gradient(cmap='RdYlGn', axis=0)
)

Проведём оценку на мультиколлинеарность, будем использовать Коэффициент инфляции дисперсии или VIF — это мера, которая помогает обнаружить степень мультиколлинеарности в множественной регрессионной модели. Он показывает, насколько увеличивается дисперсия коэффициента оценки из-за линейной зависимости с другими предикторами.

In [ ]:
# Прописываем числовые столбцы
X_num = ['маркет_актив_6_мес',
     'маркет_актив_тек_мес',
     'длительность',
     'акционные_покупки',
     'средний_просмотр_категорий_за_визит',
     'неоплаченные_продукты_штук_квартал',
     'ошибка_сервиса',
     'страниц_за_визит',
     'выручка_предыдущий_месяц',
     'выручка_препредыдущий_месяц',
     'выручка_текущий_месяц',
     'минут_предыдущий_месяц',
     'минут_текущий_месяц']

# Таблица только с числовыми столбцами
X = market_result[X_num]

# Добавляем константу
X = add_constant(X)

# Расчёт VIF для каждого предиктора
vifs = pd.DataFrame()
vifs['variable'] = X.columns
vifs['vif'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vifs

Проведите корреляционный анализ признаков в количественной шкале в итоговой таблице для моделирования. Сделайте выводы о мультиколлинеарности и при необходимости устраните её.

### Вывод

О мультиколлинеарности можно говорить если значение корреляции выше 0.9. Такой корреляции для целевого признака, покупательская активность, нет.

Проведён дополнительный анализ по VIF метрике. По её условиям о мультиколлинеарности можно говорить при значении метрики выше 5, не один параметр не отвечает данному условию.

Поэтому можно сделать вывод об отсуствии мультиколлиарности.

## Использование пайплайнов

Подготовка данных:
- Покупательская_активность - целевой признак, закодируем его значениями 0 и 1,
- Акционные_покупки - закодируем в категориальный признак.

In [ ]:
market_result['покупательская_активность'] = (
    market_result['покупательская_активность']
    .apply(lambda x: 0 if x=='прежний уровень' else 1)
    .astype(int)
)

market_result['категория_акционных_покупок'] = (
    market_result['акционные_покупки']
    .apply(lambda x: 'часто покупает по акции' if x>=0.5 else 'редко покупает по акции')
)
market_result = market_result.drop('акционные_покупки', axis=1)

# Проверим итоговую таблицу после преобразования
general_information(market_result)

In [ ]:
market_result_copy = market_result.copy()

# Сохраняем выборки
X = market_result.drop(['покупательская_активность', 'id'], axis=1)
y = market_result['покупательская_активность']

# Разделение данных на тренировочную и тестовую выборку
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    random_state=RANDOM_STATE,
    test_size=TEST_SIZE,
    stratify=y
)

# Вводим обозначения для типов исходных данных
# Нужно кодировать с помощью OneHotEncoder
ohe_columns = ['тип_сервиса',
               'разрешить_сообщать',
               'популярная_категория']

#  Нужно кодировать с помощью OrdinalEncoder
ord_columns = ['категория_акционных_покупок']

# Количественные признаки необходимо масштабировать
num_columns = ['маркет_актив_6_мес',
               'маркет_актив_тек_мес',
               'длительность',
               'средний_просмотр_категорий_за_визит',
               'неоплаченные_продукты_штук_квартал',
               'ошибка_сервиса',
               'страниц_за_визит',
               'выручка_предыдущий_месяц',
               'выручка_препредыдущий_месяц',
               'выручка_текущий_месяц',
               'минут_предыдущий_месяц',
               'минут_текущий_месяц']

In [ ]:
# Добавляем ОНЕ-кодирование в паплайн
ohe_pipe = Pipeline(
    [
        (
            'simpleImputer_ohe',
            SimpleImputer(missing_values=np.nan, strategy='most_frequent')
    ),
        (
            'ohe',
            OneHotEncoder(drop='first', handle_unknown='error')
        )
    ]
)

In [ ]:
# Добавляем Ordinal-кодирование в пайплайн
ord_pipe = Pipeline(
    [
        (
            'simpleImputer_before_ord',
            SimpleImputer(missing_values=np.nan, strategy='most_frequent')
        ),
        (
            'ord',
            OrdinalEncoder(
                categories=[
                    ['часто покупает по акции', 'редко покупает по акции']
                ],
                handle_unknown='use_encoded_value', unknown_value=np.nan
            )
        ),
        (
            'simpleImputer_after_ord',
            SimpleImputer(missing_values=np.nan, strategy='most_frequent')
        )
    ]
)

In [ ]:
# Код для паплайна подготовки данных
data_preprocessor = ColumnTransformer(
    [
        ('ohe', ohe_pipe, ohe_columns),
        ('ord', ord_pipe, ord_columns),
        ('num', MinMaxScaler(), num_columns)
    ],
    remainder='passthrough'
)

In [ ]:
# Итоговый паплайн: подготовка данных и модель
pipe_final = Pipeline(
    [
        ('preprocessor', data_preprocessor),
        ('models', DecisionTreeClassifier(random_state=RANDOM_STATE))
    ]
) 

Для модели SVC было выбрано ядро linear, а не poly т.к. затрачивалось очень много времени для выполнения кода. 

In [ ]:
param_grid = [
    # словарь для модели DecisionTreeClassifier()
    {
        'models': [DecisionTreeClassifier(random_state=RANDOM_STATE)],
        'models__max_depth': range(2, 10),
        'models__max_features': range(2, 20),
        'models__min_samples_split': range(2, 10),
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), 'passthrough']
    },
    
    # словарь для модели KNeighborsClassifier()
    {
        'models': [KNeighborsClassifier()],
        'models__n_neighbors': range(2, 20),
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), 'passthrough']
    },
    
    # словарь для модели LogisticRegression()
    {
        'models': [LogisticRegression(
            random_state=RANDOM_STATE,
            solver='liblinear',
            penalty='l1'
        )],
        'models__C': range(1, 10),
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), 'passthrough']
    },
    
    # словарь для модели SVC()
    {
        'models': [SVC(random_state=RANDOM_STATE, kernel='linear')],
        'models__degree': range(2, 20),
        'models__C': np.logspace(-2, 2, 20),
        'models__gamma': ['scale', 'auto'],
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), 'passthrough']
    }
]

In [ ]:
# Поиск лучшего решения 
# методом случайного поиска перебором гиперпараметров и методов подготовки данных
randomized_search = RandomizedSearchCV(
    pipe_final,
    param_grid,
    cv=5,
    scoring='roc_auc',
    random_state=RANDOM_STATE,
    n_jobs=-1    
)

# Обучаем
randomized_search.fit(X_train, y_train)

In [ ]:
display(f'Лучшая модель и её параметры: {randomized_search.best_estimator_}')
display(f'Метрика по кросс валидационной выборке: {randomized_search.best_score_}')

In [ ]:
# Подготовка данных
y_test_pred = randomized_search.predict(X_test)

# decision_function т.к. модель SVC
y_test_proba = randomized_search.decision_function(X_test)

display(f'Метрика ROC-AUC на тестовых данных:{roc_auc_score(y_test, y_test_proba)}')
display(f'Метрика F1 на тестовых данных:{f1_score(y_test, y_test_pred, average="macro")}')

### Вывод

Способ для поиска модели и гиперпараметров был выбран RandomizedSearchCV потому, что перед нами стоит задача найти лучшую модель идля экономии времени был выбран этот способ при следующих параметрах:
- cv=5 стандарное количество раз тестирования модели,
- scoring - стратегия оценки производительности модели была выбрана 'roc_auc' т.к. эта метрика работает при оценки сразу 4 моделей и отсуствует необходимость выберать определённые пороги отсечения и учитывает возможность дисбаланса классов.

В результате работы поплайна были обработаны 4 модели:  KNeighborsClassifier(), DecisionTreeClassifier(), LogisticRegression() и SVC(). Лучшей стала с гиперпараметрами SVC:
- С - параметр регуляризации = 1.668,
- kernel - ядро полиномиальное,
- degree - степень полимиального ядра = 2,
- random_state=100)

Метрики:
- метрика по кросс валидационной выборке: 0.897
- ROC-AUC на тестовых данных: 0.906 
- F1 на тестовых данных: 0.879

Модель выдаёт довольно хорошие метрики, что даёт нам уверенность в её работоспособности.

## Анализ важности признаков

**Подготовка данных**

In [ ]:
randomized_search.best_estimator_.named_steps['models']

In [ ]:
# Обучаем и преобразуем тренировочные данные
X_train_2 = pipe_final.named_steps['preprocessor'].fit_transform(X_train)
# Преобразуем тестовые данные
X_test_2 = pipe_final.named_steps['preprocessor'].transform(X_test)
# Выделяем имена столбцов
feature_names = pipe_final.named_steps['preprocessor'].get_feature_names_out()
# Создаём датафрейм с тестовыми даннами
X_test_2 = pd.DataFrame(X_test_2, columns=feature_names)

# Инициализируем SHAP
explainer = shap.LinearExplainer(randomized_search
                                 .best_estimator_
                                 .named_steps['models']
                                 , X_train_2)
shap_values = explainer.shap_values(X_test_2)

In [ ]:
shap.summary_plot(
    shap_values, 
    X_test_2, 
    plot_type="bar", 
    max_display=30, 
    plot_size=(15, 15) 
)

shap.summary_plot(
    shap_values, 
    X_test_2, 
    plot_type="dot", 
    max_display=30, 
    plot_size=(15, 15)  
)

plt.show()

In [ ]:
market_result.columns

### Вывод

Для модели важны признаки (интрепритация по beeswarm):
- Самыми важными признаками для сохранения покупательской активности просмотр страниц_за_визит, минут_текущий_месяц, минут_предыдущий_месяц, средний_просмотр_категорий_за_визит, маркет_актив_6_мес;
- Признаки неоплаченные_продукты_штук_квартал, разрешить_сообщать_нет, ошибка_сервиса, маркет_актив_тек_мес отрицательно влияют на сохранение покупательской активности, но их значение настолько невелико, что можно предположить, что нам необходимо искать другую причину влияющую на снижение покупательской активности.

## Сегментация покупателей

In [ ]:
# Сохраняем выборки
X_copy = market_result_copy.drop('покупательская_активность', axis=1)
y_copy = market_result_copy['покупательская_активность']

# Разделение данных на тренировочную и тестовую выборку
X_train_copy, X_test_copy, y_train_copy, y_test_copy = train_test_split(
    X_copy,
    y_copy,
    random_state=RANDOM_STATE,
    test_size=TEST_SIZE,
    stratify=y_copy
)

In [ ]:
# Копируем данные
X_train_result = X_train_copy.copy()
X_test_result = X_test_copy.copy()

# Расчитываем вероятность
X_train_result['вероятность_снижения']  = randomized_search.decision_function(X_train_copy)
X_test_result['вероятность_снижения']  = randomized_search.decision_function(X_test_copy)

# Соединяем таблицу и значения 'Вероятность_снижения'
market_result_copy = pd.concat([X_train_result, X_test_result])

# display(X_train_result.head())
# display(market_result.head())
# display(money.info())
# display(market_result.info())


# Объединение итоговой таблицы с money
money = money.set_index('id')
market_result_copy = market_result_copy.join(money, on='id')

# # Выводим общую информацию
general_information(market_result_copy)

Построим общую диаграмму рассеивания

In [ ]:
plt.figure(figsize=(10, 10))
sns.scatterplot(data=market_result_copy,
                x='вероятность_снижения',
                y='прибыль',
                s=100)
plt.title('Зависимость вероятности активности от выручки')
plt.xlabel('Вероятность снижения активности')
plt.ylabel('Прибыль')
plt.show()

Основная плотность находится в квадрате 'Вероятность снижения активности' от -3 до -1 и Прибыль от 3 до 5. Клиенты которые стараются сохранить активность приносят больше прибыли, что очевидно.

**Функция диаграммы рассеяния прибыли и вероятности снижения активности в зависимости от категориальных признаков**

In [ ]:
def market_scat(cat_columns):
    for i in cat_columns:
        plt.figure(figsize=(10, 10))
        sns.scatterplot(data=market_result_copy,
                        x='вероятность_снижения',
                        y='прибыль',
                        s=100,
                        hue=i,
                        style=i)
        plt.title('Зависимость вероятности снижения активности от выручки')
        plt.xlabel('Вероятность снижения активности')
        plt.ylabel('Прибыль')
        plt.show()

In [ ]:
# Выбераем столбцы типа object
cat_columns = list(market_result_copy.select_dtypes(include='object').columns)

# Применяем функцию
market_scat(cat_columns)

Премиумных подписок больше у клиентов которые хотят сохранить активность.

Те клиенты у которых снижается активность больше покупают по акции.

Разделим клиентов по вероятности снижения покупательской способности по нулевой границе и часто покупающих по акции.

In [ ]:
market_result_copy['сегмент'] = (
    market_result_copy
    .apply(lambda row: 'исследуемый сегмент' 
           if row['вероятность_снижения'] > 0 
           and row['категория_акционных_покупок'] == 'часто покупает по акции'
           else 'остальные пользователи',
           axis=1)
)

Построим круговые диаграммы для категориальных данных.

In [ ]:
cat_pie(market_result_copy[market_result_copy['сегмент'] == 'исследуемый сегмент'], 'тип_сервиса')

Клиентов с премиумной подпиской больше чем в таких клиентов в общем до фильтрации (шаг 3, 29% с премиумной подпиской).

In [ ]:
cat_pie(market_result_copy[market_result_copy['сегмент'] == 'исследуемый сегмент'], 'разрешить_сообщать')

Соотношение клиентов с подключенной рекламой примерно одинаково 73% против 74%.

In [ ]:
cat_pie(market_result_copy[market_result_copy['сегмент'] == 'исследуемый сегмент'], 'популярная_категория')

Товаров для детей покупают больше 35% против 25%, а мелкой бытовой техники покупают намного меньше всего 4% против 13%.

Построеним гистограммы.

In [ ]:
num_hist(market_result_copy, 'маркет_актив_6_мес', 'сегмент', 20)

Клиенты исследуемого сегмента мало получают рекламы.

In [ ]:
count_hist(market_result_copy, 'маркет_актив_тек_мес', 'сегмент')

Клиенты исследуемого сегмента мало получают рекламы.

In [ ]:
num_hist(market_result_copy, 'длительность', 'сегмент', 20)

Длительность в среднем сохраняется на всём протяжении времени, ярких всплесков нет как и провалов.

In [ ]:
count_hist(market_result_copy, 'средний_просмотр_категорий_за_визит', 'сегмент')

Просматривают 2-3 категории товаров, новых товаров видимо не ищут, просматриват только в закладках.

In [ ]:
count_hist(market_result_copy, 'неоплаченные_продукты_штук_квартал', 'сегмент')

Картина разно направленная клиентов, чем больше покупают по акции людей сохраняют товары в корзине до последнего и не оплачивают их.

In [ ]:
num_hist(market_result_copy, 'страниц_за_визит', 'сегмент', 20)

Клиенты мало просматривают страниц в магазине, ищут только нужные товары.

### Вывод

Было принято решение выделить клинтов которые покупают в основном по акции и вероятностью снижения покупательской активности т.к. за эту категорию клиетов следует поборотся и она более тругих подвержена уходу.

В ходе анализа было выявлено, что такие клиенты активно пользуются премиумной подпиской 35%. Чаще покупают товары для детей и плохо различную бытовую технику. Просматривают только нужные категории и страницы в магазине. Накапливают неоплачиваемые товары в корзине. 

Можно предположить, что такими клиентами являются клиенты с детьми и находящийся матерью в декрете. И клиенты с небольшим доходом которым нужны конкретные товары и нет средств на излишки.

Можно выдать рекомендации направить магазину больше рекламы об акциях, для повышения просмотров категорий и страниц в магазине. При возможности предлогать рассрочку в оплате товаров, для повышения лояльности клиентов, повышения товаро оборота и снижения тем самым скапливаемости товаров на складах и снижение складских расходов.

## Общий вывод

В работу поступило 3 датасета
- market_file - 13 столбцов, 1300 строк, 0 пропусков, 0 дубликатов;
- market_money - 3 столбцов, 3900 строк, 0 пропусков, 0 дубликатов;
- market_time - 3 столбцов, 2600 строк, 0 пропусков, 0 дубликатов;
- money - 2 столбцов, 1300 строк, 0 пропусков, 0 дубликатов.

В market_file и market_time отредактированы название столбцов. В money произведено разделение столбцов и проведена замена тип столбца "Прибыль" на float64.

Во всех четырёх датасетах название столбцов соотвествуют описанию.

Было выполнено:
- market_file - Исправлена опечатка в обозначении данных на "стандарт". В столбце "Длительность" данные без выбросов, в столбцах "Акционные_покупки" и "Маркет_актив_6_мес" имеются выбросы, принято решение их не удалять т.к. их достаточно много их влияние оценим на следующем шаге.
- market_money - Были удаленны два значения 106862.2 - это явный выброс и 0 - это значение удаленно по условиям исследования. Так же большое колличество выбросов которые не будут удаленны.
- market_time - Исправленна опечатка в данных на 'предыдущий_месяц'. Данные без выбросов.
- money - Значения прибыли имеют выбросы их удалять не будем.

Обобщая информацию по всем 4 датафреймам можно увидеть, что даже при снижении маркетинговой активности время пользователей в магазине не изменилось, выручка при этом расла последнии 3 месяца. 

Пользователи со сниженной покупательской активностью просматривают страниц и категорий товаров одинаково, что даёт нам понять, что они ищут товары только по акции. Они же получают всех больше рекламных уведомлений от магазина.

Пользователи с прежней активностью просматривают большое количество страниц в магазине и различные категории, и отправляют товары в корзину и оплачивают их когда в корзине становится 6 и более товаров.

По итогу 1296 клиентов сохраняло свою активность на протяжении не менее трёх месяцев.

Итоговая таблица market_result состоящая из market_file, market_money, market_time:
- 18 столбцов,
- 1296 записей,
- пропуски отсуствуют,
- дубликаты отсуствуют.

О мультиколлинеарности можно говорить если значение корреляции выше 0.9. Такой корреляции для целевого признака, покупательская активность, нет.

Проведён дополнительный анализ по VIF метрике. По её условиям о мультиколлинеарности можно говорить при значении метрики выше 5, не один параметр не отвечает данному условию.

Поэтому можно сделать вывод об отсуствии мультиколлиарности.

Способ для поиска модели и гиперпараметров был выбран RandomizedSearchCV потому, что перед нами стоит задача найти лучшую модель идля экономии времени был выбран этот способ при следующих параметрах:
- cv=5 стандарное количество раз тестирования модели,
- scoring - стратегия оценки производительности модели была выбрана 'roc_auc' т.к. эта метрика работает при оценки сразу 4 моделей и отсуствует необходимость выберать определённые пороги отсечения и учитывает возможность дисбаланса классов.

В результате работы поплайна были обработаны 4 модели:  KNeighborsClassifier(), DecisionTreeClassifier(), LogisticRegression() и SVC(). Лучшей стала с гиперпараметрами SVC:
- С - параметр регуляризации = 1.668,
- kernel - ядро полиномиальное,
- degree - степень полимиального ядра = 2,
- random_state=100)

Метрики:
- метрика по кросс валидационной выборке: 0.897
- ROC-AUC на тестовых данных: 0.906 
- F1 на тестовых данных: 0.879

Модель выдаёт довольно хорошие метрики, что даёт нам уверенность в её работоспособности.

Для модели важны признаки (интрепритация по beeswarm):
- Самыми важными признаками для сохранения покупательской активности просмотр страниц_за_визит, минут_текущий_месяц, минут_предыдущий_месяц, средний_просмотр_категорий_за_визит, маркет_актив_6_мес;
- Признаки неоплаченные_продукты_штук_квартал, разрешить_сообщать_нет, ошибка_сервиса, маркет_актив_тек_мес отрицательно влияют на сохранение покупательской активности, но их значение настолько невелико, что можно предположить, что нам необходимо искать другую причину влияющую на снижение покупательской активности.

Было принято решение выделить клинтов которые покупают в основном по акции и вероятностью снижения покупательской активности т.к. за эту категорию клиетов следует поборотся и она более тругих подвержена уходу.

В ходе анализа было выявлено, что такие клиенты активно пользуются премиумной подпиской 35%. Чаще покупают товары для детей и плохо различную бытовую технику. Просматривают только нужные категории и страницы в магазине. Накапливают неоплачиваемые товары в корзине. 

Можно предположить, что такими клиентами являются клиенты с детьми и находящийся матерью в декрете. И клиенты с небольшим доходом которым нужны конкретные товары и нет средств на излишки.

Можно выдать рекомендации направить магазину больше рекламы об акциях, для повышения просмотров категорий и страниц в магазине. При возможности предлогать рассрочку в оплате товаров, для повышения лояльности клиентов, повышения товаро оборота и снижения тем самым скапливаемости товаров на складах и снижение складских расходов.